In [43]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [44]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [62]:
wiki_api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)
wiki_tool.name = "wikiQueryRun" 

In [46]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [47]:
loader=WebBaseLoader('https://m.dailyhunt.in/news/india/english/for+you?mode=pwa&launch=true')
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
vector_db=FAISS.from_documents(documents=documents,embedding=embeddings)
retriver=vector_db.as_retriever()

In [48]:
retriver

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBE0B51F90>, search_kwargs={})

In [49]:
from langchain.tools.retriever import create_retriever_tool
retriver_news_tool = create_retriever_tool(
    retriver,
    'DailyNews',  # Ensure name is valid
    "Search for information about daily news updates."
)

In [64]:
retriver_news_tool.name = "dailyNewsSearch"
retriver_news_tool

Tool(name='dailyNewsSearch', description='Search for information about daily news updates.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001DB83C827A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBE0B51F90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001DB84029900>, retriever=VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001DBE0B51F90>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), docume

In [65]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv_tool.name = "arxivQueryRun"

In [72]:
tools = [wiki_tool, arxiv_tool, retriver_news_tool]

In [59]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="models/gemini-pro")

from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [67]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [75]:
# Before invoking the agent
input_data = {
    "input": "Rahul Gandhi",
    "intermediate_steps": []  # or any other expected structure
}

print("Input data:", input_data)  # Debugging output

response = agent.invoke(input_data)
print("Response:", response)  # Check the structure of the response


Input data: {'input': 'Rahul Gandhi', 'intermediate_steps': []}


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.tools[0].function_declarations[0].name: Invalid function name. Must start with a letter or an underscore. Must be alphameric (a-z, A-Z, 0-9), underscores (_), dots (.) or dashes (-), with a maximum length of 64.
* GenerateContentRequest.tools[0].function_declarations[1].name: Invalid function name. Must start with a letter or an underscore. Must be alphameric (a-z, A-Z, 0-9), underscores (_), dots (.) or dashes (-), with a maximum length of 64.
* GenerateContentRequest.tools[0].function_declarations[2].name: Invalid function name. Must start with a letter or an underscore. Must be alphameric (a-z, A-Z, 0-9), underscores (_), dots (.) or dashes (-), with a maximum length of 64.
